In [1]:
import os

In [2]:
%pwd

'c:\\Users\\midem\\Facial-Emotion-Recognition-Using-MLflow-and-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\midem\\Facial-Emotion-Recognition-Using-MLflow-and-DVC'

In [ ]:
# To be used to update entity (src/emotionRecognition/entity/config_entity.py)
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
# To be used to update configuration manager (src/emotionRecognition/config/configuration.py)
from emotionRecognition.constants import *
from emotionRecognition.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [ ]:
# To be used to update the data ingestion component (src/emotionRecognition/components/data_ingestion.py
import os
import urllib.request as request
import zipfile
from emotionRecognition import logger
from emotionRecognition.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
# To be used to update the data ingestion pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-16 17:17:45,409: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-16 17:17:45,415: INFO: common: yaml file: params.yaml loaded successfully]


[2024-11-16 17:17:45,419: INFO: common: created directory at: artifacts]
[2024-11-16 17:17:45,420: INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-16 17:18:51,307: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
x-amz-id-2: piGUmmNc8xotTNNOD8YlPqO95hJMeSeUaWXmRCP7Lji+QAb479OVFsIlc2VlEq9wxl1UwRouwlM=
x-amz-request-id: 6FF9JH1QS3N2VQZR
Date: Sat, 16 Nov 2024 17:17:46 GMT
Last-Modified: Fri, 15 Nov 2024 21:33:57 GMT
ETag: "b797c3210f284fb1a15d6d94b3112f21-27"
x-amz-server-side-encryption: AES256
Accept-Ranges: bytes
Content-Type: application/zip
Content-Length: 454218927
Server: AmazonS3
Connection: close

]
